# Bayesian Time Series

*Note*: There have been relatively large (and exciting!) updates to `pymc3` since the Bayesian statistics lectures. This includes the PyMC3 team officially taking over the Theano project and renaming it `aesara` -- Any of the functionality that we had previously used should be effectively the same.

In [ ]:
# aesara is missing a couple pieces of functionality
# still so we still use theano...
# import aesara
# import aesara.tensor as at
import theano as aesara
import theano.tensor as at
import arviz as az
import matplotlib.pyplot as plt
import numba
import numpy as np
import pymc3 as pm

%matplotlib inline

## Refresher

Let's start spend a few minutes refreshing our knowledge of Bayesian methods.

We use the following notation:

* $y$ represents data
* $\theta$ represents parameters

**Bayes Law**:

$$P(\theta | y) = \frac{P(\theta) P(y | \theta)}{P(\theta)}$$


**Prior**: $P(\theta)$

The prior represents your individual beliefs about the probability of certain combinations of parameters having generated the data of interest.

**Likelihood**: $P(y | \theta)$

The likelihood function is a function that maps values of the parameters, $\theta \in \Theta$ and observations, $y \in \mathcal{Y}$, into the probability of having observed $y$ given $\theta$.

**Posterior**: $P(\theta | y)$

The posterior combines the prior and likelihood to specify what your beliefs about what the probability of certain combinations parameters having generated the data (after observing data).

**Markov Chain Monte-Carlo (MCMC)**:

A class of methods used to simulate samples from the posterior distribution.

We have previously discussed the Metropolis-Hastings algorithm, but, it is dated, and, as we mentioned, modern software (like `pymc3`) typically samples using the no u-turn sampler.

If you want a graphical refresher for how these algorithms work, we refer you back to this (excellent) [online tool](https://chi-feng.github.io/mcmc-demo/app.html) developed by [Chi Feng](https://github.com/chi-feng)

**Why do I like Bayesian models?**

* Partially driven by beliefs of people that I respect and learned from
* Partially that the Bayesian notion of probability (beliefs about the likelihood of something occurring rather than frequencies with which something occurs) makes more sense to me
* Significantly motivated by their ability to quantify uncertainty

## Advanced PyMC3 Usage: Aesara

**What is Aesara?**

>Aesara is not a programming language in the normal sense because you write a program in Python that builds expressions for Aesara. Still it is like a programming language in the sense that you have to
>
>* declare variables (a, b) and give their types
>* build expressions for how to put those variables together
>* compile expression graphs to functions in order to use them for computation.
>
> It is good to think of `aesara.function` as the interface to a compiler which builds a callable object from a purely symbolic graph. One of Aesara’s most important features is that `aesara.function` can optimize a graph and even compile some or all of it into native machine instructions.
>
>\- _From [Aesara documentation](https://aesara.readthedocs.io/en/latest/index.html)_

>Aesara’s compiler applies many optimizations of varying complexity to these symbolic expressions. These optimizations include, but are not limited to:
>
>* use of GPU for computations
>* constant folding
>* merging of similar subgraphs, to avoid redundant calculation
>* arithmetic simplification (e.g. x*y/x -> y, --x -> x)
>* inserting efficient BLAS operations (e.g. GEMM) in a variety of contexts
>* using memory aliasing to avoid calculation
>* using inplace operations wherever it does not interfere with aliasing
>* loop fusion for elementwise sub-expressions
>* improvements to numerical stability (e.g. \log(1+\exp(x)) and \log(\sum_i \exp(x[i])))
>* and [many more](https://aesara.readthedocs.io/en/latest/optimizations.html#optimizations)
>
>\- _From [Aesara documentation](https://aesara.readthedocs.io/en/latest/index.html)_

> Fun fact: The library that Aesara is based on, Theano, was written at the LISA lab to support rapid development of efficient machine learning algorithms. Theano was named after the Greek mathematician, who may have been Pythagoras’ wife. Aesara is an alleged daughter of Pythagoras and Theano.
>
>\- _From [Aesara documentation](https://aesara.readthedocs.io/en/latest/index.html)_

**Example program**

In [ ]:
# declare two symbolic floating-point scalars
a = at.dscalar()
b = at.dscalar()

# create a simple expression
c = a + b

# convert the expression into a callable object that takes `(a, b)`
# values as input and computes a value for `c`
f = aesara.function([a, b], c)

# bind 1.5 to 'a', 2.5 to 'b', and evaluate 'c'
assert 4.0 == f(1.5, 2.5)

**Scan**

One of the reasons that we're talking about Aesara today is because many time series models require us to write loops within our PyMC3 models. This is slow because the (potential) type instability associated with these loops prevents Aesara from being able to make the optimizations that make PyMC3 so fast!

In [ ]:
k = at.iscalar("k")
A = at.vector("A")

# Symbolic description of the result
result, updates = aesara.scan(
    fn=lambda prior_result, A: prior_result * A,
    outputs_info=at.ones_like(A),
    non_sequences=A, n_steps=k
)

# We only care about A**k, but scan has provided us with A**1 through A**k.
# Discard the values that we don't care about. Scan is smart enough to
# notice this and not waste memory saving them.
final_result = result[-1]

# compiled function that returns A**k
power = aesara.function(inputs=[A, k], outputs=final_result, updates=updates)

In [ ]:
print(power(range(10), 2))

In [ ]:
print(power(range(10), 3))

We will see more examples soon -- The important thing is understanding that `scan` performs the "compilation" step and the theoretical outputs are stored in `result`

## "Warm-up": Revisiting AR/MA models

We begin by revisiting the models we discussed previously

**Autoregressive**

\begin{align*}
  y_{t+1} &= \rho_1 y_t + \rho_2 y_{t-1} + \sigma \varepsilon_{t+1}
\end{align*}

In [ ]:
@numba.jit(nopython=True)
def simulate_ar2(T, rho_1, rho_2, sigma):
    # Allocate space
    out = np.zeros(T)
    for t in range(T):
        if t < 2:
            out[t] = 0
        else:
            out[t] = rho_1*out[t-1] + rho_2*out[t-2] + sigma*np.random.randn()
    return out

In [ ]:
ar_data = simulate_ar2(250, 0.7, -0.2, 0.1)

In [ ]:
plt.plot(ar_data)

In [ ]:
def one_step(eps, ym1, ym2, rho_1, rho_2, sigma):
    return rho_1*ym1 + rho_2*ym2 + sigma*eps

y = at.vector("y")
eps = at.vector("eps")
rho_1 = at.dscalar()
rho_2 = at.dscalar()
sigma = at.dscalar()

results, updates = aesara.scan(
    one_step, sequences=[
        dict(input=eps, taps=[0])
    ], outputs_info=dict(
        initial=y, taps=[-1, -2]
    ),
    non_sequences=[rho_1, rho_2, sigma]
)

ar_stepper = aesara.function(
    inputs=[eps, y, rho_1, rho_2, sigma],
    outputs=results, updates=updates
)

In [ ]:
ar_scan = ar_stepper(np.random.randn(100), np.zeros(2), 0.9, -0.2, 0.1)

plt.plot(ar_scan)

"Computing the model by hand"

In [ ]:
nobs = ar_data.shape[0]
ar_fl = pm.Model()

with ar_fl:
    # Data
    data_fl = pm.Data("data", ar_data)

    # Parameters
    rho_1 = pm.Uniform("rho_1", -1, 1)
    rho_2 = pm.Uniform("rho_2", -1, 1)
    sigma = pm.HalfNormal("sigma", 3)

    # Update via loop
    for t in range(2, nobs):
        obs = pm.Normal(
            f"obs_{t}", rho_1*data_fl[t-1] + rho_2*data_fl[t-2],
            sigma, observed=data_fl[t]
        )


In [ ]:
# %%time
# Not going to run this live -- It takes a few minutes to compile
# the model and initialize the sampler... Gets slower as number
# of observations increase...

# with ar_fl:
#     trace_fl = pm.sample(750, return_inferencedata=False)
#     az.plot_trace(trace_fl, var_names=["rho_1", "rho_2", "sigma"])

Computed via scan

In [ ]:
ar_bh = pm.Model()

with ar_bh:
    # Data
    data_bh = pm.Data("data", ar_data)

    # Parameters
    rho_1 = pm.Uniform("rho_1", -1, 1)
    rho_2 = pm.Uniform("rho_2", -1, 1)
    sigma = pm.HalfNormal("sigma", 3)

    # Update via scan
    def one_step_mu(ytm1, ytm2, rho_1, rho_2):
        return rho_1*ytm1 + rho_2*ytm2

    _mu, update = aesara.scan(
        one_step_mu,
        sequences=[dict(input=data_bh, taps=[-1, -2])],
        non_sequences=[rho_1, rho_2]
    )

    obs = pm.Normal("obs", _mu, sigma, observed=data_bh[2:])

In [ ]:
%%time

with ar_bh:
    trace_bh = pm.sample(750, return_inferencedata=False)
    az.plot_trace(trace_bh)

Vectorized

In [ ]:
ar_vec = pm.Model()

with ar_vec:
    # Data
    data_vec = pm.Data("data", ar_data)

    # Parameters
    rho_1 = pm.Uniform("rho_1", -1, 1)
    rho_2 = pm.Uniform("rho_2", -1, 1)
    sigma = pm.HalfNormal("sigma", 3)

    # Compute mean
    mu = rho_1*data_bh[1:-1] + rho_2*data_bh[0:-2]
    obs = pm.Normal("obs", mu, sigma, observed=data_vec[2:])

In [ ]:
%%time

with ar_vec:
    trace_vec = pm.sample(750, return_inferencedata=False)
    az.plot_trace(trace_vec)

Using PyMC3 functionality

In [ ]:
ar_pymc3 = pm.Model()

with ar_pymc3:
    # Data
    data_pymc3 = pm.Data("data", ar_data)

    # Parameters
    rho_1 = pm.Uniform("rho_1", -1, 1)
    rho_2 = pm.Uniform("rho_2", -1, 1)
    sigma = pm.HalfNormal("sigma", 3)

    obs = pm.AR("observed", [rho_1, rho_2], sigma, observed=data_pymc3)

In [ ]:
%%time

with ar_pymc3:
    trace_pymc3 = pm.sample(750, return_inferencedata=False)
    az.plot_trace(trace_pymc3)

## Moving average models

\begin{align*}
  y_{t+1} &= \phi_1 \varepsilon_{t} + \phi_2 \varepsilon_{t-1} + \varepsilon_{t+1}
\end{align*}

In [ ]:
@numba.jit(nopython=True)
def simulate_ma2(T, phi_1, phi_2, sigma):
    # Allocate space
    epsilons = sigma*np.random.randn(T+2)
    out = np.zeros(T)
    for t in range(T):
        out[t] = (
            phi_1*epsilons[t+2-1] + 
            phi_2*epsilons[t+2-2] + 
            epsilons[t+2]
        )

    return out, epsilons

In [ ]:
ma_data, ma_epsilons = simulate_ma2(50, 0.75, 0.25, 0.25)

In [ ]:
plt.plot(ma_data)

How do we build this Bayesian model?

* We need to compute the $\{\varepsilon_t\}$
* Given $y_0$, $\varepsilon_{-1}$, and $\varepsilon_{-2}$, we can compute $\varepsilon_0$... Given $y_t$, $\varepsilon_{t-1}$, and $\varepsilon_{t-2}$, then we can compute $\varepsilon_t$...
* Let's place priors over $\varepsilon_{t-1}$ and $\varepsilon_{t-2}$

In [ ]:
ma_scan = pm.Model()

with ma_scan:
    # Data
    data_scan = pm.Data("data", ma_data)

    # Parameters
    phi_1 = pm.Normal("phi_1", 0, 2.5)
    phi_2 = pm.Normal("phi_2", 0, 2.5)
    sigma = pm.HalfNormal("sigma", 5)

    # Unobserved epsilons
    eps_m2m1 = pm.Normal("eps_m2m1", 0, sigma/2, shape=2)

    # Update via scan
    def compute_eps_t(yt, epsm2, epsm1, phi_2, phi_1):
        return (yt - phi_1*epsm1 - phi_2*epsm2)

    _eps, update = aesara.scan(
        compute_eps_t,
        sequences=[dict(input=data_scan, taps=[0])],
        outputs_info=[
            dict(initial=eps_m2m1, taps=[-2, -1])
        ],
        non_sequences=[phi_2, phi_1]
    )
    epsilons = pm.Deterministic("epsilons", _eps)

    obs0 = pm.Normal(
        "obs0", phi_1*eps_m2m1[1] + phi_2*eps_m2m1[0],
        sigma, observed=data_scan[0]
    )
    obs1 = pm.Normal(
        "obs1", phi_1*epsilons[0] + phi_2*eps_m2m1[1],
        sigma, observed=data_scan[1]
    )
    obs = pm.Normal(
        "obs", phi_1*epsilons[1:-1] + phi_2*epsilons[0:-2], sigma,
        observed=data_scan[2:]
    )

In [ ]:
with ma_scan:
    trace_ma = pm.sample(
        1000, return_inferencedata=True,
        tune=1500, target_accept=0.95
    )
    az.plot_trace(trace_ma)

In [ ]:
posterior_epsilons = trace_ma["posterior"]["epsilons"]

posterior_epsilons_quantiles = posterior_epsilons.quantile(
    [0.05, 0.5, 0.95], dim=["chain", "draw"]
).values

In [ ]:
T = ma_epsilons.shape[0]

fig, ax = plt.subplots()

# Realized epsilons
ax.plot(ma_epsilons, color="k", linestyle="-", linewidth=1.5)

# Quantiles and median
ax.plot(
    np.arange(2, T),
    posterior_epsilons_quantiles[1, :],
    color="ForestGreen", linestyle="--", linewidth=1.5
)
ax.fill_between(
    np.arange(2, T),
    posterior_epsilons_quantiles[0, :],
    posterior_epsilons_quantiles[2, :],
    color="ForestGreen", alpha=0.5
)

ax.set_title(r"Realized vs Estimated $\varepsilon$s")

## Bayesian decomposable time series models

We spoke about additive decompositions in a previous lecture. It is a method that aims to describe a time series data using the sum of certain components (trend, seasonal, etc...).

One relatively recent tool that has been released that does Bayesian time series decomposition is an open source library developed by Facebook called "Prophet"


**Prophet**

[Prophet](https://facebook.github.io/prophet/) is an excellent piece of software that implements many sensible defaults (especially when being applied to data similar to things that Facebook might be interested in).

However, we also want to acknowledge the PyMC3-based ([`pm-prophet`](https://github.com/luke14free/pm-prophet)). The benefit to this package over the original Prophet is that they have left the flexibility to modify the priors and other components of the model!

### The model

Prophet (and the PyMC3 based PM-Prophet) build a model of the following format:

$$y_t = g(t) + s(t) + h(t) + \varepsilon_t$$

where

* $y_t$ is the observation at period $t$
* $\varepsilon_t$ is noise
* $g(t)$ is a trend component
* $s(t)$ is a collection of seasonal components
* $h(t)$ is a holiday component


**Trend component**

Original Prophet model supports either:

* Piecewise-linear trends with specified changepoints
* Piecewise-logistic growth

In both cases, subsets of the parameters are allowed to change as time passes on to account for time-varying trends

**Seasonal component**

Uses Fourier series to approximate generic (smooth) seasonality in the data

$$s(t) = \sum_{n=1}^N \left(a_n \cos\left(\frac{2 \pi n t}{P}\right) + b_n \sin\left(\frac{2 \pi n t}{P}\right) \right)$$

Need to pick:

* $P$: The period to approximate (i.e. 7 for weekly seasonality, 365.25 for yearly, etc...)
* $N$: The number of Fourier series elements to include -- Higher $N$ allows for more flexible fitting but raises risk of overfitting

**Holiday component**

Each holiday is given a window and a magnitude parameter. Let $D_h = [h - j, h + j]$ then a day is assigned an indicator of 1 if it falls in the window $D_h$ and the magnitude of the effect is measured as $k$.

### Applications

[Prophet paper](https://peerj.com/preprints/3190/)

## Fun blogposts that are worth reading

* [Sorry ARIMA, but I’m Going Bayesian](https://multithreaded.stitchfix.com/blog/2016/04/21/forget-arima/)
* [Tis the Season... to be Bayesian!](https://multithreaded.stitchfix.com/blog/2020/12/16/tis-the-season-to-be-bayesian/)
* [Talk by one of main developers of Prophet](https://www.youtube.com/watch?v=pOYAXv15r3A&ab_channel=LanderAnalytics)